# Analytique de l'apprentissage

## Collecte et extraction des données 

## Exemple de la plateforme Open edX

Carnet web IPython (Jupyter Notebook) en complément du **«Chapitre 6 – Vers une IENA pilotée par les données » ** du livre : **« Vers une nouvelle ingénierie des environnements numériques d’apprentissage »**, auteurs du chapitre, Claude Coulombe & Valéry Psyché, éditeurs: Josiane Basque, Gilbert Paquette, licence code source libre Apache

## Plateforme Open edX

Plateforme de cours en ligne ouverts et massifs CLOM (en anglais MOOC: massive open online course) 

https://open.edx.org/about-open-edx/

## La collecte des données

La première étape du travail d’analytique est la collecte des données.

Comme exemple, nous collecterons les réponses de la question 3 du questionnaire socio-démogaphique. À la question « Dans quel pays habitez-vous ? », l’apprenant sélectionnait sa réponse dans un menu déroulant qui était mémorisée dans la base de données.

Ci-dessous une requête SQL pour collecter les données brutes sur la question 3 et les sauvegarder dans un fichier `IHPQ_Profil_Q3_brut.csv`. Le choix du format `.csv`, signifiant 'Comma Separated Value', est très pratique car compatible avec la plupert des outils de l'écosystème Python pour la science des données. 

La structure de données contenant la réponse pour la question 3 est mémorisée dans le champ « state » pour chaque apprenant «student_id» du module identifié (`module_id`) par ` i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82`.

**Note**: Le code SQL est donné à titre indicatif mais ne fonctionne pas dans ce carnet Python.

```SQL
SELECT 
    edxapp.courseware_studentmodule.student_id, 
    edxapp.courseware_studentmodule.module_id, 
    edxapp.courseware_studentmodule.state
FROM 
    edxapp.courseware_studentmodule 
WHERE
    edxapp.courseware_studentmodule.module_id=     
        "i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82" 
INTO OUTFILE 
    '/home/data_usagers/IHPQ_Profil_Q3_brut.csv' 
FIELDS TERMINATED BY ';' ENCLOSED BY '"' LINES TERMINATED BY '\n' ;
```

Nous avons délibérément choisi de limiter l’utilisation du SQL à la collecte des données brutes et à leur sauvegarde dans des fichiers tabulaires.

## Le prétraitement des données

Tous les traitements subséquents (ou prétraitements) seront effectués en Python dans une console ou dans des carnets Jupyter iPython pratiques. Ce choix a été motivé par la plus grande souplesse des outils Python et par une meilleure maîtrise des outils Python que des outils SQL.

Ces traitements impliqueront la manipulation de tableaux de données appelés « DataFrames » avec les outils de la bibliothèque Pandas qui est utilisée pour le pré-traitement et la manipulation des données [McKinney, 2010].  

### Lecture et chargement des données brutes :
Commençons par charger les données du fichier des données brutes, `IHPQ_Profil_Q3_Brut.csv` dans un tableau de données Pandas (dataframe) `IHPQ_Profil_Q3_DF`.

In [24]:
import pandas as pd
path_to_datafile = "AA_data/"

In [25]:
IHPQ_Profil_Q3_DF = pd.read_csv (path_to_datafile+"IHPQ_Profil_Q3_Brut.csv", 
                                delimiter=",",
                                na_values=[""],
                                encoding='utf-8')

L'opération de lecture `pd.read_csv(...)` retourne `IHPQ_Profil_Q3_DF`, un Dataframe ou tableau de données analogue à une feuille de tableur où chaque ligne représente un exemplaire de données (observations ou exemples) et chaque colonne représente la valeur d'un attribut (ou caractéristique). 

Avec la commande `.head()` demandons à Pandas d'afficher les cinq premières lignes du tableau de données `IHPQ_Profil_Q3_DF` en configurant Pandas pour afficher jusqu’à 1000 caractères par champ ou colonne.

In [26]:
pd.options.display.max_colwidth = 1000
IHPQ_Profil_Q3_DF.head ()

,student_id,module_id,state
0,13,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Canada - Qc""}}"
1,4818,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Ha\u00efti""}, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}}"
2,3535,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""\u00c9tats-Unis""}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}}"
3,1044,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""France""}}"
4,3977,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Canada - Qc""}}"


#### Modification / uniformisation des données avec une expression régulière (caractères accentués)

Nous remarquons problème d’encodage dans les noms de pays, par exemple « Haïti » est encodé « Ha\u00efti » ce qui sera facilement corrigé en appliquant une petite fonction de correction à chaque cellule `'state'`. 

In [27]:
IHPQ_Profil_Q3_DF['state'] = IHPQ_Profil_Q3_DF['state'].apply(lambda text: text.replace("\\/", "/").encode().decode('unicode_escape')  if type(text) is str  else text)
IHPQ_Profil_Q3_DF.head()

,student_id,module_id,state
0,13,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Canada - Qc""}}"
1,4818,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Haïti""}, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}}"
2,3535,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""États-Unis""}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}}"
3,1044,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""France""}}"
4,3977,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Canada - Qc""}}"


### Extraction d'un attribut au moyen d'une expression régulière

L’étape suivante est l’extraction du nom du pays de la chaîne de caractères contenue dans la colonne « “state” ».
du tableau de données (dataframe), nous utiliserons une expression régulière. Une expression régulière est un formalisme qui décrit la forme générale d’une chaîne de caractères. Une expression régulière est utilisée pour rechercher des chaînes de caractères de même forme dans un texte. Une fois les chaînes trouvées, on peut leur appliquer un traitement, comme un ajout, une modification ou une suppression [WIKIPÉDIA, Regular expression]. 

Une expression régulière est un formalisme qui décrit la forme générale d’une chaîne de caractères. Une expression régulière est utilisée pour rechercher des chaînes de caractères de même forme dans un texte. Une fois les chaînes trouvées, on peut leur appliquer un traitement, comme un ajout, une modification ou une suppression [WIKIPÉDIA, Regular expression]. 

Le résultat de l’extraction des différentes valeurs est affecté dans la nouvelle colonne « “pays” » du tableau des données « IHPQ_Profil_Q3_DF ». Cette colonne du tableau de données représente les différentes valeurs l’attribut « “pays” ».


In [28]:
IHPQ_Profil_Q3_DF['pays'] = IHPQ_Profil_Q3_DF['state'].str.extract('\":\s*\"([A-Za-ü\s*\-\.\s*]*)\"')
IHPQ_Profil_Q3_DF.head()

,student_id,module_id,state,pays
0,13,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Canada - Qc""}}",Canada - Qc
1,4818,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Haïti""}, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}}",Haïti
2,3535,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""États-Unis""}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}}",États-Unis
3,1044,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""France""}}",France
4,3977,i4x://TELUQ/TUL2/problem/0704643ff2e943dbae4bffcae0bb7d82,"{""correct_map"": {}, ""seed"": 1, ""done"": null, ""input_state"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": {}}, ""student_answers"": {""i4x-TELUQ-TUL2-problem-0704643ff2e943dbae4bffcae0bb7d82_2_1"": ""Canada - Qc""}}",Canada - Qc


In [29]:
IHPQ_Profil_Q3_DF["pays"].head()

0    Canada - Qc
1          Haïti
2     États-Unis
3         France
4    Canada - Qc
Name: pays, dtype: object

## Ressources utiles

1) StackOverflow - https://stackoverflow.com

2) Pandas -https://pandas.pydata.org/pandas-docs/stable/getting_started/tutorials.html

3) Expressions régulières - http://regex101.com/ - https://www.regular-expressions.info/tutorial.html

4) Matplotlib - https://matplotlib.org/3.1.1/tutorials/index.html

